# Demonstration of Memory with a Long Short-Term Memory Network in Python


## Problem Representatio

### One Hot Encoding
We will use a one hot encoding to represent the learning problem for the LSTM.

That is, each input and output value will be represented as a binary vector with 5 elements, because the alphabet of the problem is 5 unique values.

For example, the 5 values of [0, 1, 2, 3, 4] are represented as the following 5 binary vectors:

0: \[1, 0, 0, 0, 0 \]

1: \[0, 1, 0, 0, 0 \]

2: \[0, 0, 1, 0, 0 \]

3: \[0, 0, 0, 1, 0 \]

4: \[0, 0, 0, 0, 1 \]


In [1]:
def encode(pattern, n_unique):
    """
    - binary encode an input pattern
    - return a list of binary vectors for each value in sequence
    """
    encoded = list()
    for value in pattern:
        row = [0.0 for x in range(n_unique)]
        row[value] = 1.0
        encoded.append(row)
        
    return encoded

In [2]:
seq1 = [3, 0, 1, 2, 3]
encoded = encode(seq1, 5)
for vector in encoded:
    print(vector)

[0.0, 0.0, 0.0, 1.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 1.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 1.0, 0.0]


### Input-Output Pairs

In [3]:
# create input/output pairs of encoded vectors, returns X, y
def to_xy_pairs(encoded):
	X,y = list(),list()
	for i in range(1, len(encoded)):
		X.append(encoded[i-1])
		y.append(encoded[i])
	return X, y

In [4]:
X, y = to_xy_pairs(encoded)
for i in range(len(X)):
    print(X[i], y[i])

[0.0, 0.0, 0.0, 1.0, 0.0] [1.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0] [0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0, 0.0] [0.0, 0.0, 1.0, 0.0, 0.0]
[0.0, 0.0, 1.0, 0.0, 0.0] [0.0, 0.0, 0.0, 1.0, 0.0]


### Reshape Data
The final step is to reshape the data so that it can be used by the LSTM network directly.

The Keras LSTM expects input patterns (X) as a three-dimensional NumPy array with the dimensions [samples, timesteps, features].

In the case of one sequence of input data, the dimensions will be [4, 1, 5] because we have 4 rows of data, 1 time step for each row, and 5 columns in each row.

In [5]:
def to_lstm_dataset(sequence, n_unique):
    """
    - takes a sequence and the size of the sequence alphabet as input  
    - and returns an X and y dataset ready for use with an LSTM. 
    - It performs the required conversions of the sequence to a 
    one-hot encoding 
    - and to input-output pairs before reshaping the data.
    """
    # one hot encode
    encoded = encode(sequence, n_unique)
    
    # convert to in/out patterns
    X,y = to_xy_pairs(encoded)
    
    # convert to LSTM friendly format
    dfX, dfy = DataFrame(X), DataFrame(y)
    lstmX = dfX.values
    lstmX = lstmX.reshape(lstmX.shape[0], 1, lstmX.shape[1])
    lstmY = dfy.values
    return lstmX, lstmY

In [14]:
from pandas import DataFrame
seq1 = [3, 0, 1, 2, 3]
seq2 = [4, 0, 1, 2, 4]
n_unique = len(set(seq1 + seq2))

seq1X, seq1Y = to_lstm_dataset(seq1, n_unique)
seq2X, seq2Y = to_lstm_dataset(seq2, n_unique)

print("seq1X: {} \n seq1Y: {} ".format(seq1X, seq1Y))
print("seq2X: {} \n seq2Y: {} ".format(seq2X, seq2Y))

seq1X: [[[0. 0. 0. 1. 0.]]

 [[1. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0.]]

 [[0. 0. 1. 0. 0.]]] 
 seq1Y: [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]] 
seq2X: [[[0. 0. 0. 0. 1.]]

 [[1. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0.]]

 [[0. 0. 1. 0. 0.]]] 
 seq2Y: [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]] 


## Learn Sequences with an LSTM

In [1]:
from pandas import DataFrame
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# binary encode an input pattern, return a list of binary vectors
def encode(pattern, n_unique):
	encoded = list()
	for value in pattern:
		row = [0.0 for x in range(n_unique)]
		row[value] = 1.0
		encoded.append(row)
	return encoded

# create input/output pairs of encoded vectors, returns X, y
def to_xy_pairs(encoded):
	X,y = list(),list()
	for i in range(1, len(encoded)):
		X.append(encoded[i-1])
		y.append(encoded[i])
	return X, y

# convert sequence to x/y pairs ready for use with an LSTM
def to_lstm_dataset(sequence, n_unique):
	# one hot encode
	encoded = encode(sequence, n_unique)
	# convert to in/out patterns
	X,y = to_xy_pairs(encoded)
	# convert to LSTM friendly format
	dfX, dfy = DataFrame(X), DataFrame(y)
	lstmX = dfX.values
	lstmX = lstmX.reshape(lstmX.shape[0], 1, lstmX.shape[1])
	lstmY = dfy.values
	return lstmX, lstmY

# define sequences
seq1 = [3, 0, 1, 2, 3]
seq2 = [4, 0, 1, 2, 4]
# convert sequences into required data format
n_unique = len(set(seq1 + seq2))
seq1X, seq1Y = to_lstm_dataset(seq1, n_unique)
seq2X, seq2Y = to_lstm_dataset(seq2, n_unique)

# define LSTM configuration
n_neurons = 20
n_batch = 1
n_epoch = 250
n_features = n_unique

# create LSTM
"""
Keras LSTMs are not stateful between batches by default. 
We can make them stateful by setting the stateful argument on the 
LSTM layer to True and managing the training epochs manually to 
ensure that the internal state of the LSTM is reset after each 
sequence.
"""
model = Sequential()
model.add(LSTM(n_neurons, batch_input_shape=(n_batch, 1, n_features), 
               stateful=True))
model.add(Dense(n_unique, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')

# train LSTM
for i in range(n_epoch):
	model.fit(seq1X, seq1Y, epochs=1, batch_size=n_batch, verbose=1, 
              shuffle=False)
    # Within one epoch we can fit the model on each sequence, 
    # being sure to reset state after each sequence.
	model.reset_states()
	model.fit(seq2X, seq2Y, epochs=1, batch_size=n_batch, verbose=0, 
              shuffle=False)
	model.reset_states()

# test LSTM on sequence 1
print('Sequence 1')
result = model.predict_classes(seq1X, batch_size=n_batch, verbose=0)
model.reset_states()
for i in range(len(result)):
	print('X=%.1f y=%.1f, yhat=%.1f' % (seq1[i], seq1[i+1], result[i]))

# test LSTM on sequence 2
print('Sequence 2')
result = model.predict_classes(seq2X, batch_size=n_batch, verbose=0)
model.reset_states()
for i in range(len(result)):
	print('X=%.1f y=%.1f, yhat=%.1f' % (seq2[i], seq2[i+1], result[i]))

Using TensorFlow backend.


Epoch 1/1
4/4 [==============================] - 1s 309ms/step - loss: 0.6980
Epoch 1/1
4/4 [==============================] - 0s 1ms/step - loss: 0.6921
Epoch 1/1
4/4 [==============================] - 0s 1ms/step - loss: 0.6868
Epoch 1/1
4/4 [==============================] - 0s 1ms/step - loss: 0.6817
Epoch 1/1
4/4 [==============================] - 0s 2ms/step - loss: 0.6765
Epoch 1/1
4/4 [==============================] - 0s 1ms/step - loss: 0.6713
Epoch 1/1
4/4 [==============================] - 0s 1ms/step - loss: 0.6660
Epoch 1/1
4/4 [==============================] - 0s 1ms/step - loss: 0.6605
Epoch 1/1
4/4 [==============================] - 0s 1ms/step - loss: 0.6549
Epoch 1/1
4/4 [==============================] - 0s 992us/step - loss: 0.6489
Epoch 1/1
4/4 [==============================] - 0s 2ms/step - loss: 0.6427
Epoch 1/1
4/4 [==============================] - 0s 1ms/step - loss: 0.6362
Epoch 1/1
4/4 [==============================] - 0s 1ms/step - loss: 0.6293
Epoch 1/

4/4 [==============================] - 0s 2ms/step - loss: 0.3691
Epoch 1/1
4/4 [==============================] - 0s 2ms/step - loss: 0.3676
Epoch 1/1
4/4 [==============================] - 0s 1ms/step - loss: 0.3659
Epoch 1/1
4/4 [==============================] - 0s 1ms/step - loss: 0.3643
Epoch 1/1
4/4 [==============================] - 0s 959us/step - loss: 0.3627
Epoch 1/1
4/4 [==============================] - 0s 1ms/step - loss: 0.3612
Epoch 1/1
4/4 [==============================] - 0s 1ms/step - loss: 0.3595
Epoch 1/1
4/4 [==============================] - 0s 1ms/step - loss: 0.3579
Epoch 1/1
4/4 [==============================] - 0s 1ms/step - loss: 0.3562
Epoch 1/1
4/4 [==============================] - 0s 962us/step - loss: 0.3544
Epoch 1/1
4/4 [==============================] - 0s 966us/step - loss: 0.3522
Epoch 1/1
4/4 [==============================] - 0s 1ms/step - loss: 0.3500
Epoch 1/1
4/4 [==============================] - 0s 983us/step - loss: 0.3479
Epoch 1/1
4/4 

4/4 [==============================] - 0s 2ms/step - loss: 0.2072
Epoch 1/1
4/4 [==============================] - 0s 1ms/step - loss: 0.2061
Epoch 1/1
4/4 [==============================] - 0s 1ms/step - loss: 0.2051
Epoch 1/1
4/4 [==============================] - 0s 1ms/step - loss: 0.2041
Epoch 1/1
4/4 [==============================] - 0s 1ms/step - loss: 0.2031
Epoch 1/1
4/4 [==============================] - 0s 991us/step - loss: 0.2021
Epoch 1/1
4/4 [==============================] - 0s 1ms/step - loss: 0.2012
Epoch 1/1
4/4 [==============================] - 0s 2ms/step - loss: 0.2003
Epoch 1/1
4/4 [==============================] - 0s 1ms/step - loss: 0.1993
Epoch 1/1
4/4 [==============================] - 0s 991us/step - loss: 0.1983
Epoch 1/1
4/4 [==============================] - 0s 1ms/step - loss: 0.1974
Epoch 1/1
4/4 [==============================] - 0s 985us/step - loss: 0.1965
Epoch 1/1
4/4 [==============================] - 0s 1ms/step - loss: 0.1956
Epoch 1/1
4/4 [=